In [1]:
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer
import torch

/home/suraj/anaconda3/envs/lams/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = LlamaTokenizer.from_pretrained(MODEL_NAME)
model = LlamaForCausalLM.from_pretrained(
    MODEL_NAME,
    return_dict=True,
    # load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards: 100%|██████████| 2/2 [10:13<00:00, 306.61s/it]


In [3]:
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 4096,
  "pad_token_id": 0,
  "temperature": 0.6,
  "top_p": 0.9,
  "transformers_version": "4.33.2"
}

In [4]:
model = model.eval()


In [65]:
prompt = "My name is Vikas"
# prompt = "Names of my friends are: Anmol, Aryan, Joel. How many friends do I have?"
prompt = "Do you recall the names of my three friends?"


In [66]:
encoding = tokenizer(prompt, return_tensors="pt").to(model.device)

In [67]:
print(encoding)
print(encoding.input_ids)
print(encoding.input_ids.shape)

{'input_ids': tensor([[    1,  1938,   366, 17386,   278,  2983,   310,   590,  2211,  7875,
         29973]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}
tensor([[    1,  1938,   366, 17386,   278,  2983,   310,   590,  2211,  7875,
         29973]], device='cuda:0')
torch.Size([1, 11])


In [68]:
# %%time
with torch.no_grad():
    outputs = model.generate(
        **encoding,
        max_new_tokens=256,
        temperature=0.0001,
        generation_config=generation_config,
    )

In [69]:
print("len", len(outputs))
print("shape", outputs.shape)



len 1
shape torch.Size([1, 54])


In [70]:

output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(output_text)

Do you recall the names of my three friends?

I'm glad you asked! They are John, Sarah, and... (pauses and looks at you with a mischievous grin) ...Moo! (winks)


In [22]:
answer_tokens = outputs[:, encoding.input_ids.shape[1] :]


print("Original shape", outputs.shape)
print("Inpute Shape", encoding.input_ids.shape)
print("New shape", answer_tokens.shape)



output_text = tokenizer.decode(answer_tokens[0], skip_special_tokens=True).strip()
print(output_text)

Original shape torch.Size([1, 214])
Inpute Shape torch.Size([1, 10])
New shape torch.Size([1, 204])
?

Vikas is a digital marketing expert and a social media influencer. He is the founder of @vrmvikas, a digital marketing agency that helps businesses grow their online presence through social media marketing, SEO, content marketing, and more.
Vikas is also a popular social media influencer with over 100k followers on Instagram. He shares tips and insights on digital marketing, social media marketing, and entrepreneurship through his posts and stories.
In addition to his work as a digital marketing agency owner and social media influencer, Vikas is also a sought-after speaker and trainer. He has delivered talks and workshops on digital marketing and social media marketing at various events and conferences.
Overall, Vikas is a highly respected figure in the digital marketing industry, known for his expertise and passion for helping businesses succeed online.


In [10]:
def format_prompt(prompt: str, system_prompt: str) -> str:
    return f"""
{system_prompt}
{prompt}
""".strip()

In [11]:
def generate_response(prompt: str, max_new_tokens: int = 128) -> str:
    encoding = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.inference_mode():
        outputs = model.generate(
            **encoding,
            max_new_tokens=max_new_tokens,
            temperature=0.001,
            generation_config=generation_config,
        )
    answer_tokens = outputs[:, encoding.input_ids.shape[1] :]
    return tokenizer.decode(answer_tokens[0], skip_special_tokens=True)
     

In [54]:
SYSTEM_PROMPT = """
""".strip()

In [13]:
prompt = """
Write an email to a director flaunting about your movie career
""".strip()
response = generate_response(format_prompt(prompt, SYSTEM_PROMPT), max_new_tokens=512)
print(response)

In [55]:
format_prompt(prompt, SYSTEM_PROMPT)

'Vikas Verm'

In [52]:
prompt = """
Fake News is becoming a huge problem today.
""".strip()
responses = []
for _ in range(10):
    response = generate_response(format_prompt(prompt, SYSTEM_PROMPT), max_new_tokens=200)
    print(response)
    SYSTEM_PROMPT = "You know that " + response

and achievements.
Subject: My illustrious career in cinema
Dear [Director's Name],
I hope this email finds you in the best of spirits. As a fellow filmmaker, I am writing to share with you my immense pride and satisfaction in the illustrious career that I have had the privilege of pursuing in the world of cinema.
Over the years, I have had the good fortune of being a part of some of the most iconic and memorable films in Indian cinema. From my debut in the 1970s to the present day, I have consistently delivered performances that have left audiences in awe and inspired generations of actors to follow in my footsteps.
My filmography boasts of a diverse range of roles, from the intense and brooding anti-hero in "Deewar" to the charismatic and larger-than-life figure in "Sholay". I have essayed complex and nuanced characters with ease, bringing to life some of the most memorable characters in Indian cinema. My performances have been widely acclaimed and have earned me numerous awards and a

In [ ]:
SYSTEM_PROMPT = """
Write a response that answers the request according to the conversation history below.
"""